In [2]:
# -*- coding: utf-8 -*-
"""
Created on Thu Mar 15 20:56:04 2018

@author: corey


Cleaning up the  Yelp Dataset

"""
import pandas as pd
from pprint import pprint
import numpy as np
import json
import censusgeocode as cg

df = pd.read_csv('LA_County_restaurants.csv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
df.head()

df = df[df['city'] == "Los Angeles"]

,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,...,review_count,transactions,url,Latitude,Longitude,city,zipcode,category1,category2,category3
35,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.0047569274902, 'longitude': -1...",(323) 561-1675,54768.500159,fettes-schwein-food-truck-los-angeles-3,https://s3-media3.fl.yelpcdn.com/bphoto/3LOYvP...,False,"{'address1': '', 'address2': '', 'address3': '...",Fettes Schwein Food Truck,1.323561e+10,...,14,[],https://www.yelp.com/biz/fettes-schwein-food-t...,-118.259079,34.004757,Los Angeles,90011,Food Trucks,Hot Dogs,Burgers
45,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.10161, 'longitude': -118.30206}",(323) 477-6764,53442.069549,the-tropic-truck-los-angeles-5,https://s3-media2.fl.yelpcdn.com/bphoto/Fq9EuT...,False,"{'address1': None, 'address2': None, 'address3...",The Tropic Truck,1.323478e+10,...,50,[],https://www.yelp.com/biz/the-tropic-truck-los-...,-118.302060,34.101610,Los Angeles,90027,Food Trucks,Caribbean,Caterers
47,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.1463062227897, 'longitude': -1...",(818) 373-9767,64863.573440,okamoto-kitchen-los-angeles-2,https://s3-media3.fl.yelpcdn.com/bphoto/9bs7YT...,False,"{'address1': None, 'address2': None, 'address3...",Okamoto Kitchen,1.818374e+10,...,168,"['delivery', 'pickup', 'restaurant_reservation']",https://www.yelp.com/biz/okamoto-kitchen-los-a...,-118.425397,34.146306,Los Angeles,91423,Food Trucks,Japanese,Caterers
53,"[{'alias': 'filipino', 'title': 'Filipino'}, {...","{'latitude': 34.0569114685059, 'longitude': -1...",(888) 907-2248,50926.777789,white-rabbit-truck-los-angeles-4,https://s3-media1.fl.yelpcdn.com/bphoto/-_twM9...,False,"{'address1': None, 'address2': None, 'address3...",White Rabbit Truck,1.888907e+10,...,569,[],https://www.yelp.com/biz/white-rabbit-truck-lo...,-118.243950,34.056911,Los Angeles,90012,Filipino,Food Trucks,Caterers
72,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.21332, 'longitude': -118.35861}",(818) 282-4301,58069.947811,shawarma-and-falafel-palace-food-truck-los-ang...,https://s3-media4.fl.yelpcdn.com/bphoto/jKyVhe...,False,"{'address1': '7782 San Fernando Rd', 'address2...",Shawarma and Falafel Palace Food Truck,1.818282e+10,...,1,[],https://www.yelp.com/biz/shawarma-and-falafel-...,-118.358610,34.213320,Los Angeles,91352,Food Trucks,Middle Eastern,NaN
964,"[{'alias': 'foodtrucks', 'title': 'Food Trucks...","{'latitude': 34.029109954834, 'longitude': -11...",(562) 794-6484,51357.806229,urban-espresso-los-angeles-6,https://s3-media3.fl.yelpcdn.com/bphoto/bUKrBN...,False,"{'address1': '', 'address2': None, 'address3':...",Urban Espresso,1.562795e+10,...,25,[],https://www.yelp.com/biz/urban-espresso-los-an...,-118.283066,34.029110,Los Angeles,90007,Food Trucks,Coffee & Tea,Breakfast & Brunch
1473,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 34.13904, 'longitude': -118.20416}",(323) 255-5111,12323.773752,cafe-beaujolais-los-angeles,https://s3-media2.fl.yelpcdn.com/bphoto/MZ0MtL...,False,"{'address1': '1712 Colorado Blvd', 'address2':...",Cafe Beaujolais,1.323256e+10,...,1261,[],https://www.yelp.com/biz/cafe-beaujolais-los-a...,-118.204160,34.139040,Los Angeles,90041,French,NaN,NaN
1480,"[{'alias': 'breakfast_brunch', 'title': 'Break...","{'latitude': 34.12768, 'longitude': -118.21722}",(323) 474-6420,14061.604868,rockn-egg-cafe-los-angeles-3,https://s3-media2.fl.yelpcdn.com/bphoto/YwVMqf...,False,"{'address1': '4616 Eagle Rock Blvd', 'address2...",Rock'n Egg Cafe,1.323475e+10,...,33,[],https://www.yelp.com/biz/rockn-egg-cafe-los-an...,-118.217220,34.127680,Los Angeles,90041,Breakfast & Brunch,Diners,Sandwiches
1487,"[{'alias': 'vegan', 'title': 'Vegan'}]","{'latitude': 34.1343840632986, 'longitude': -1...",(323) 775-6614,13367.367100,the-vegan-hooligans-los-angeles,https://s3-media1.fl.yelpcdn.com/bphoto/cXOsCI...,False,"{'address1': '4862 Eagle Rock B

In [100]:
# this variable is for testing extract_location
loc_sample = "{'address1': '349 N State College Blvd', 'address2': None, 'address3': None, 'city': 'Fullerton', 'zip_code': '92831', 'country': 'US', 'state': 'CA', 'display_address': ['349 N State College Blvd', 'Fullerton, CA 92831']}"


def extract_category(df):
    category1 = []
    category2 = []
    category3 = []
    for x in df['categories']:
        json_rec = json.loads(x.replace("'", '"'))
        category1.append(json_rec[0]['title'])
        try:
            category2.append(json_rec[1]['title'])
        except:
            category2.append('')
        try:
            category3.append(json_rec[2]['title'])
        except:
            category3.append('')
    df['category1'] = category1
    df['category2'] = category2
    df['category3'] = category3

extract_category(df)

In [87]:
#pprint(loc_sample)


def extract_location(df):
    '''
    breaks yelp's location data along the key that we want and then
    returns it as a new dataframe
    '''
    cities = []
    zip_codes = []
    streets = []
    #print(df['location'])
    #print(json.dumps(df['location']))
    placeholder = []
    for row in df['location']:
        try:
            address = row.split("'display_address': ['")[1].rstrip("']}").split("', '")
            city = address[1].split(",")[0]
            #print(city)
            cities.append(city)
            street = address[0]
            print(street, city)
            streets.append(address[0])
            #print(address)
        except:
            print('error: \n')
            print(row)
            cities.append('none')
            streets.append('none')
        
        try:
            zip_code = row.split("'zip_code': ")[1].split(",")[0].strip("'")
        except:
            zip_code = np.nan
            
        zip_codes.append(zip_code)
    df['city'] = cities
    df['street'] = streets
    df['zipcode'] = zip_codes
        

       
extract_location(df)    

3165 Yorba Linda Blvd Fullerton
349 N State College Blvd Fullerton
660 N Rose Dr Placentia
2931 Nutwood Ave Fullerton
1325 E Chapman Ave Fullerton
711 N Placentia Ave Fullerton
3203 N Glassell St Orange
114 W Commonwealth Ave Fullerton
305 N State College Blvd Fullerton
2323 E Chapman Ave Ste B
512 N State College Blvd Fullerton
2905 E Miraloma Ave Ste 3
616 W Chapman Ave Placentia
3405 E Imperial Hwy Brea
305 N State College Blvd Fullerton
201 S Bradford Ave Placentia
2279 N Tustin St Orange
1956 N Placentia Ave Fullerton
440 S Anaheim Blvd Ste 104
733 N Placentia Ave Fullerton
1450 N Kraemer Blvd Placentia
1060 E Imperial Hwy Brea
141 E Commonwealth Ave Fullerton
202 W Lincoln Ave Anaheim
2445 E Imperial Hwy Ste M
229 E Commonwealth Ave Fullerton
2555 E Chapman Ave Fullerton
1071 N Tustin Ave Ste 110
2445 E Imperial Hwy Ste F
401 S Magnolia Ave Anaheim
3458 E Orangethorpe Ave Anaheim
1343 E Chapman Ave Fullerton
907 E Yorba Linda Blvd Placentia
374 S Main St Orange
184 S Harbor Blvd 

1313 S Harbor Blvd Anaheim
26676 Portola Pkwy Ste D
2136 Colorado Blvd Los Angeles
440 S Anaheim Blvd No 209-B
10708 Los Alamitos Blvd Los Alamitos
Buena Vista St Anaheim
121 Mc Fadden Pl Newport Beach
13221 Garden Grove Blvd Garden Grove
14258 Newport Ave Tustin
10043 Artesia Pl Bellflower
6740 Greenleaf Ave Whittier
12681 Harbor Blvd Garden Grove
619 N Azusa Ave Azusa
3757 Riverside Dr Ste A
400 W Disney Way Ste 337
46 E Huntington Dr Arcadia
1834 Puente Ave Baldwin Park
7511 Edinger Ave Huntington Beach
228 W 2nd St Pomona
129 E Arrow Hwy Glendora
11332 Washington Blvd Whittier
6339 Etiwanda Ave Mira Loma
5363 E Arrow Hwy Montclair
2114 E Lincoln Ave Anaheim
5555 Hamner Ave Norco
417 S Brookhurst St Anaheim
53 E Union St Pasadena
30 E Orangethorpe Ave Anaheim
10602 Lower Azusa Rd El Monte
1822 Chico Ave South El Monte
2000 E Gene Autry Way Anaheim
11761 Edinger Ave Fountain Valley
820 W 19th St Costa Mesa
16701 Pacific Coast Hwy Sunset Beach
1060 Walnut Ave Tustin
16025 Gale Ave Ind

814 W 19th St Costa Mesa
4523 Gage Ave Bell
3583 N Los Coyotes Diagonal Long Beach
2839 S Diamond Bar Blvd Diamond Bar
1821 W Commonwealth Ave Ste H
194 Marina Dr Ste 101
5621 N Figueroa St Los Angeles
116 E Chapman Ave Orange
4100 Donald Douglas Dr Long Beach
11001 S Central Ave Ontario
9469 Foothill Blvd Rancho Cucamonga
2438 E Katella Ave Anaheim
11265 Topaz Ave Fountain Valley
22722 Ctr Dr Ste A
5100 E 2nd St Long Beach
2493 Park Ave Ste 41
14001 Newport Ave Tustin
1542 W Holt Blvd Ontario
101 Main St Seal Beach
2001 Westcliff Dr Ste 100
error: 

{'address1': '1776 S Disneyland Dr', 'address2': '', 'address3': "Disney's California Adventure", 'city': 'Anaheim', 'zip_code': '92802', 'country': 'US', 'state': 'CA', 'display_address': ['1776 S Disneyland Dr', "Disney's California Adventure", 'Anaheim, CA 92802']}
8228 Firestone Blvd Downey
2142 E 4th St Long Beach
2421 E Artesia Blvd Long Beach
2471 Whittier Blvd Los Angeles
28 W Main St Alhambra
8082 Adams Ave Huntington Beach
4702 W

In [101]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11684 entries, 0 to 11683
Data columns (total 22 columns):
categories       11684 non-null object
coordinates      11684 non-null object
display_phone    11338 non-null object
distance         11684 non-null float64
id               11684 non-null object
image_url        11575 non-null object
is_closed        11684 non-null bool
location         11684 non-null object
name             11684 non-null object
phone            11338 non-null float64
price            11192 non-null object
rating           11684 non-null float64
review_count     11684 non-null int64
transactions     11684 non-null object
url              11684 non-null object
Latitude         11684 non-null float64
Longitude        11684 non-null float64
city             11684 non-null object
zipcode          11663 non-null object
category1        11684 non-null object
category2        11684 non-null object
category3        11684 non-null object
dtypes: bool(1), float64(5), in

In [1]:
import pandas as pd
from pprint import pprint
import numpy as np
import json
import censusgeocode as cg

df = pd.read_csv('LA_County_restaurants.csv', sep='\t', encoding='utf-8', index_col='Unnamed: 0')
df = df[df['city'] == "Los Angeles"]


results = []
for index,row in df.iterrows():
    lat = round(row['Latitude'],5)
    print(lat)
    lon = round(row['Longitude'],5)
    print(lon)
    
    try:
        response = cg.coordinates(x=lat, y=lon)
        tract = response['2010 Census Blocks'][0]['TRACT']
        print(tract)
        results.append(f"6037{tract}")
    except:
        results.append(np.nan)
        pass
    


-118.25908
34.00476
228600
-118.30206
34.10161
190520
-118.4254
34.14631
141102
-118.24395
34.05691
207400
-118.35861
34.21332
122200
-118.28307
34.02911
224700
-118.20416
34.13904
181400
-118.21722
34.12768
183402
-118.21535
34.13438
181600
-118.2026
34.13977
181000
-118.20055
34.1394
181500
-118.21645
34.12974
183402
-118.19124
34.11827
183620
-118.19577
34.11937
183220
-118.19845
34.13898
181500
-118.2128
34.13896
181400
-118.22111
34.14177
181000
-118.17902
34.11883
183101
-118.1854
34.12325
-118.21667
34.13107
181600
-118.2124
34.13889
181400
-118.20543
34.13955
-118.21534
34.13606
181400
-118.20586
34.12157
183401
-118.20165
34.1206
183300
-118.21187
34.13897
181400
-118.20615
34.12104
183401
-118.21231
34.1389
181400
-118.18721
34.12733
183221
-118.2101
34.13906
181400
-118.20654
34.12111
183401
-118.18941
34.11844
183220
-118.21535
34.13438
181600
-118.21793
34.12378
183402
-118.20095
34.13911
181500
-118.22058
34.12407
186201
-118.20448
34.13917
181400
-118.19609
34.11077
1838

-118.20162
34.02642
-118.17319
34.04043
-118.12415
33.97494
532304
-118.1583
34.09536
201120
-118.18684
34.06307
201402
-118.19222
34.03618
531101
-118.1868
34.03631
531101
-118.16216
34.03429
530500
-118.21526
34.03421
204700
-118.17836
34.08461
201401
-118.21958
34.00953
532400
-118.2192
34.04708
206032
-118.19257
34.02876
203900
-118.17768
34.0841
201503
-118.15041
34.01933
531701
-118.19287
34.10994
183610
-118.21826
34.04677
206032
-118.16598
34.02163
531602
-118.21168
34.04869
203600
-118.21935
34.0268
205110
-118.19328
34.1095
183610
-118.1644
34.01748
531702
-118.19256
34.11012
183610
-118.19246
34.01855
204920
-118.20423
34.03692
204300
-118.20027
34.03533
204120
-118.21404
34.02142
205120
-118.21074
34.07405
199201
-118.19404
34.03676
203900
-118.2044
34.02179
204820
-118.19828
34.0317
203900
-118.16193
34.03466
530500
-118.20119
34.02803
204810
-118.21471
34.07869
199201
-118.21984
34.05257
203500
-118.16593
34.01816
531602
-118.19421
34.04899
203200
-118.20065
34.06868
1991

267402
-118.43493
34.04738
267200
-118.45456
34.04455
267501
-118.44325
34.04108
267700
-118.44199
34.03884
267700
-118.46496
33.9998
273200
-118.44964
33.95781
278102
-118.44912
34.04683
267300
-118.46088
34.04207
267402
-118.5257
34.04695
262706
-118.43248
33.99648
702803
-118.44303
34.02884
271300
-118.445
34.05957
265202
-118.464
34.0543
-118.46741
34.05305
264301
-118.47449
33.98773
273502
-118.44798
34.0623
265202
-118.44589
34.04382
267700
-118.44234
34.03893
267700
-118.47331
33.98617
273502
-118.4774
33.99672
273402
-118.46334
34.05351
264301
-118.44326
34.04002
267700
-118.44084
34.0379
267700
-118.44942
34.04605
267300
-118.46377
34.0543
264103
-118.47171
33.99218
273300
-118.44312
34.04677
267200
-118.46312
34.0481
264301
-118.44286
34.04026
267700
-118.443
34.03944
267700
-118.44783
34.047
267300
-118.46863
34.07761
262200
-118.46924
33.99126
273502
-118.46291
33.99922
273100
-118.46434
34.04619
267402
-118.46312
34.0481
264301
-118.43804
34.03976
267800
-118.46476
34.054


267700
-118.44312
34.05735
265510
-118.43936
34.03467
271200
-118.44784
33.95969
278102
-118.46342
34.04763
264301
-118.45941
34.04174
267501
-118.43932
34.02254
271300
-118.44788
34.05758
701100
-118.44094
34.05458
265510
-118.43808
34.03967
267800
-118.44175
34.03941
267700
-118.42924
34.02774
271702
-118.46919
34.07716
262200
-118.45255
34.04558
267300
-118.4462
34.06023
265202
-118.44833
34.04531
267300
-118.46008
33.99084
273600
-118.45439
34.044
267501
-118.44319
33.99297
702803
-118.44672
34.06098
265202
-118.44765
34.06351
265202
-118.4609
34.04219
267402
-118.44443
34.12762
261200
-118.44333
34.0483
265520
-118.44774
34.03026
271200
-118.44696
34.06017
265305
-118.44561
34.06162
265202
-118.45662
34.04342
267501
-118.44813
34.0631
265202
-118.43375
34.04673
267200
-118.42929
34.02743
271702
-118.44815
34.04625
267300
-118.45006
34.0704
265301
-118.46601
34.04488
267402
-118.44347
34.04099
267700
-118.48
33.994
273402
-118.45961
34.04162
267501
-118.44283
34.03296
271200
-117.9

189101
-118.33589
34.10185
190202
-118.32792
34.10141
190700
-118.26677
34.10768
195100
-118.33701
34.10551
189702
-118.3473
34.08364
192001
-118.34584
34.0833
192001
-118.27791
34.09072
195400
-118.39211
34.14347
143602
-118.34342
34.0327
219700
-118.33856
34.10074
190700
-118.37363
34.0728
214800
-118.36114
34.07974
214400
-118.30728
34.0693
211410
-118.329
34.09944
190700
-118.31971
34.09761
190902
-118.34915
34.08367
192002
-118.37475
34.16576
125320
-118.34928
34.07642
214000
-118.30997
34.09874
190510
-118.36902
34.17241
124300
-118.2751
34.08556
195803
-118.36187
34.07897
194500
-118.28806
34.06404
211120
-118.28148
34.14852
980009
-118.30151
34.07649
192520
-118.25957
34.10516
187300
-118.30513
34.10188
190402
-118.26869
34.02991
224020
-118.33578
34.10086
190700
-118.38352
34.13964
143800
-118.30937
34.07888
192420
-118.27935
34.07414
208610
-118.34528
34.1013
189904
-118.3107
34.06193
211704
-118.23702
34.05972
207102
-118.36263
34.07225
214600
-118.27773
34.09019
195902
-118

186401
-118.3768
34.07418
214902
-118.30946
34.06414
211704
-118.36147
34.08064
194401
-118.30865
34.10227
190402
-118.2562
34.04722
207710
-118.23561
34.06217
207103
-118.33103
34.10159
190700
-118.25153
34.07402
197700
-118.24507
34.04883
207302
-118.32643
34.09592
190801
-118.35252
34.09845
189905
-118.24501
34.051
207302
-118.3357
34.09756
190700
-118.29964
34.0474
213310
-118.30994
34.05599
212620
-118.31294
34.10132
190510
-118.27233
34.10625
195100
-118.34368
34.07214
214100
-118.28039
34.09215
195901
-118.34915
34.09813
189905
-118.30289
34.10133
190520
-118.34999
34.0833
192002
-118.35761
34.04927
217100
-118.28727
34.10759
189101
-118.36373
34.11473
194102
-118.3087
34.05461
212502
-118.27521
34.02829
224420
-118.34989
34.06484
215101
-118.25557
34.04697
207710
-118.29609
34.09567
-118.32254
34.04768
212900
-118.29378
34.06356
212101
-118.29164
34.10382
189101
-118.23951
34.0614
207102
-118.34439
34.08403
192001
-118.30819
34.09105
191120
-118.24864
34.05063
207301
-118.34173

239201
-118.33904
34.00986
236100
-118.2919
34.03588
221601
-118.42508
33.98535
275311
-118.29161
34.03849
221602
-118.3114
33.96002
238100
-118.41651
33.97663
275602
-118.25068
34.04653
207301
-118.26535
33.9247
241002
-118.25495
34.04879
207710
-118.40696
33.94189
980028
-118.24481
33.9601
535102
-118.30886
34.0401
221302
-118.41127
34.02876
269904
-118.24709
34.04773
207302
-118.26024
34.04538
207710
-118.31832
33.99349
234600
-118.25073
34.04663
207301
-118.25776
34.04285
207900
-118.37818
33.96002
277100
-118.2787
34.0249
224700
-118.25578
33.95989
535102
-118.26342
34.04036
207900
-118.3344
34.01581
234200
-118.28236
33.91674
291110
-118.25438
34.03682
226002
-118.30865
33.96891
237900
-118.23495
34.04265
206031
-118.41914
33.96001
276500
-118.2545
34.01875
227020
-118.23858
34.04669
206200
-118.24708
34.04716
207302
-118.25366
34.04669
207301
-118.42138
34.04402
267902
-118.33641
34.01076
236100
-118.38159
33.9462
277200
-118.25979
34.04757
207710
-118.25914
34.04193
207900
-118

In [7]:
result = {'Counties': [{'OID': 275901063468976, 'STATE': '06', 'FUNCSTAT': 'A', 'AREAWATER': 1794659470, 'NAME': 'Los Angeles County', 'LSADC': '06', 'CENTLON': '-118.2617650', 'BASENAME': 'Los Angeles', 'INTPTLAT': '+34.1963983', 'COUNTYCC': 'H1', 'MTFCC': 'G4020', 'COUNTY': '037', 'GEOID': '06037', 'CENTLAT': '+34.1957768', 'INTPTLON': '-118.2618616', 'AREALAND': 10510687541, 'COUNTYNS': '00277283', 'OBJECTID': 398, 'CENT': (-118.261765, 34.1957768), 'INTPT': (-118.2618616, 34.1963983)}], 'Census Tracts': [{'OID': 207901115289836, 'STATE': '06', 'FUNCSTAT': 'S', 'NAME': 'Census Tract 1993', 'AREAWATER': 40785, 'LSADC': 'CT', 'CENTLON': '-118.1992140', 'BASENAME': '1993', 'INTPTLAT': '+34.0941911', 'MTFCC': 'G5020', 'COUNTY': '037', 'GEOID': '06037199300', 'CENTLAT': '+34.0926249', 'INTPTLON': '-118.2003961', 'AREALAND': 2550540, 'OBJECTID': 6991, 'TRACT': '199300', 'CENT': (-118.199214, 34.0926249), 'INTPT': (-118.2003961, 34.0941911)}], '2010 Census Blocks': [{'BLKGRP': '1', 'OID': 210404056348637, 'FUNCSTAT': 'S', 'STATE': '06', 'AREAWATER': 0, 'NAME': 'Block 1000', 'SUFFIX': '', 'LSADC': 'BK', 'CENTLON': '-118.1942484', 'LWBLKTYP': 'L', 'BASENAME': '1000', 'BLOCK': '1000', 'INTPTLAT': '+34.1001521', 'MTFCC': 'G5040', 'COUNTY': '037', 'GEOID': '060371993001000', 'CENTLAT': '+34.1001521', 'INTPTLON': '-118.1942484', 'AREALAND': 428518, 'OBJECTID': 3068287, 'TRACT': '199300', 'CENT': (-118.1942484, 34.1001521), 'INTPT': (-118.1942484, 34.1001521)}], 'States': [{'OID': 2749018475066, 'STATE': '06', 'FUNCSTAT': 'A', 'NAME': 'California', 'AREAWATER': 20484627967, 'LSADC': '00', 'CENTLON': '-119.5277460', 'STUSAB': 'CA', 'BASENAME': 'California', 'INTPTLAT': '+37.1551773', 'DIVISION': '9', 'MTFCC': 'G4000', 'STATENS': '01779778', 'GEOID': '06', 'CENTLAT': '+37.1547352', 'INTPTLON': '-119.5434183', 'REGION': '4', 'AREALAND': 403483191859, 'OBJECTID': 14, 'CENT': (-119.527746, 37.1547352), 'INTPT': (-119.5434183, 37.1551773)}]}
# 6037101110 
print(f"6037{result['2010 Census Blocks'][0]['TRACT']}")

6037199300


In [4]:
df['CensusTract'] = results

In [5]:
df.to_csv('LA_County_Resturants_CensusTracts.csv', encoding='utf-8')